# Import and Login To Google Drive
You must have all the files in your drive, at the root level

In [0]:
import tensorflow as tf
import numpy as np
import networkx as nx
import math

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

# Read In the File and Make Graph in G

In [0]:
import re
lineFinder = re.compile('(\d+\s+\d+.*\n)')
# Options com-youtube.ungraph.txt  ca-HepTh.txt roadNet-CA.txt
with open('/content/drive/My Drive/ca-HepTh.txt', 'r') as f:
   file = f.read()
singleLines = list(map(lambda x: np.fromstring(x, dtype=int, sep=' '), lineFinder.findall(file)))
G = nx.from_edgelist(np.column_stack(singleLines).T)
G = nx.convert_node_labels_to_integers(G)

# Eigenvalues/vectors With Spare Part

In [0]:
from scipy.sparse import lil_matrix
adjacency_matrix = lil_matrix((len(G.nodes()), len(G.nodes())))
#diag_matrix = lil_matrix((len(G.nodes()), len(G.nodes())))
normal_laplacian = lil_matrix((len(G.nodes()), len(G.nodes())))
print(adjacency_matrix.shape)
G = nx.convert_node_labels_to_integers(G)
for edge in G.edges():
  adjacency_matrix[edge[0],edge[1]] = -1
  adjacency_matrix[edge[1],edge[0]] = -1
  adjacency_matrix[edge[0],edge[0]] +=1.0
  adjacency_matrix[edge[1],edge[1]] +=1.0
print(adjacency_matrix.getnnz())
for edge in G.edges():
  degree_v1 = adjacency_matrix[edge[0],edge[0]]
  degree_v2 = adjacency_matrix[edge[1],edge[1]]
  total_deg = -1/(math.sqrt(degree_v1*degree_v2))
  normal_laplacian[edge[0],edge[1]] = total_deg
  normal_laplacian[edge[1],edge[0]] = total_deg
  normal_laplacian[edge[0],edge[0]] = 1.0
  normal_laplacian[edge[1],edge[1]] = 1.0
print(normal_laplacian.getnnz())

(9877, 9877)
61823
61823


In [0]:
randomCommunities = []
import random
for i in range(1,12):
  for j in range(0,2**i)
    randomAllocation.append(set())
  print(randomAllocation)
  for j in range(len(connected_communities[0].nodes)):
    randomAllocation[random.randint(0,9)].add(i)
  print(randomAllocation)
  print(mod_score(randomCommunities,connected_communities[0]))

SyntaxError: ignored

In [0]:
from scipy.sparse.linalg import eigsh
eigenvalues, eigenvectors = eigsh(normal_laplacian, k=3, which='LA')

In [0]:
print(eigenvalues, eigenvectors)

[2.1199614  2.36349067 2.79871327] [[-9.15061315e-07  1.07356044e-05 -6.65583015e-06]
 [ 6.63902806e-06  2.99867080e-04 -1.64248510e-05]
 [-3.28998816e-08 -4.69438469e-04  1.54694368e-04]
 ...
 [-2.62571494e-17  8.82373560e-18 -5.69587718e-18]
 [ 9.73583763e-18 -1.95277320e-18 -7.77809827e-20]
 [ 2.06321026e-17 -9.25887552e-18  3.35054177e-19]]


# Upload and Import Ground Truth If Testing That

In [0]:
# This is wrong doesn't input full adj list
#GTruth = nx.read_adjlist('/content/drive/My Drive/com-youtube.all.cmty.txt',delimiter='\t',nodetype=int)


# Running a Built In Algo

In [0]:
connected_communities = sorted(list(nx.connected_component_subgraphs(G)),key=lambda x: -1*len(x.nodes()));

In [0]:
import networkx.algorithms.community as algos
values = {}

from collections import Counter
from networkx.exception import NetworkXError
from networkx.algorithms.components import is_connected
from networkx.utils import groups
from networkx.utils import not_implemented_for
from random import shuffle
from random import choice
def alternate_async_flow(G, k, max_iter=100, seed=None):
  max_density = 1.0
  vertices = list(G)
  shuffle(vertices)
  communities = {n: i for i, n in enumerate(vertices[:k])}
  com_to_numvertices = {}
  for vertex in communities.keys():
      com_to_numvertices[communities[vertex]] = 1
  # Set up control variables and start iterating
  iter_count = 0
  cont = True
  while cont:
      cont = False
      iter_count += 1
      # Loop over all vertices in graph in a random order
      vertices = list(G)
      shuffle(vertices)
      for vertex in vertices:
          # Updating rule
          com_counter = Counter()
          # Take into account self vertex community
          try:
              com_counter.update({communities[vertex]:
                                  1})
          except KeyError:
              pass
          # Gather neighbour vertex communities
          for v in G[vertex]:
              try:
                  com_counter.update({communities[v]:
                                      1})
              except KeyError:
                  continue
                  
          #My Code, modifys com_counter
          computed = {}
          for key in com_counter.keys():
            computed[key] = 1 / (1 - math.log(com_counter[key]) + math.log(com_to_numvertices[key]))
          # Check which is the community with highest density
          new_com = -1
          if len(computed.keys()) > 0:
              max_freq = max(computed.values())
              best_communities = [com for com, freq in computed.items()
                                  if (max_freq - freq) < 0.0001]
              # If actual vertex com in best communities, it is preserved
              try:
                  if communities[vertex] in best_communities:
                      new_com = communities[vertex]
              except KeyError:
                  pass
              # If vertex community changes...
              if new_com == -1:
                  # Set flag of non-convergence
                  cont = True
                  # Randomly chose a new community from candidates
                  new_com = choice(best_communities)
                  # Update previous community status
                  try:
                      com_to_numvertices[communities[vertex]] -= 1
                  except KeyError:
                      pass
                  # Update new community status
                  communities[vertex] = new_com
                  com_to_numvertices[communities[vertex]] += 1
      # If maximum iterations reached --> output actual results
      if iter_count > max_iter:
          break
  # Return results by grouping communities as list of vertices
  return iter(groups(communities).values())

import time

totalScoreOriginal = {}
totalScoreNew = {}
totalTimeOriginal = {}
totalTimeNew = {}
numTrials = 2
scalingFactor = 2
maxKTrials = 10
for k in (scalingFactor**i for i in range(maxKTrials)):
  result = 0
  result2 = 0
  timeStart =0
  timeStart2 = 0
  for _ in range(0, numTrials):
    timeStart -= time.time()
    result_t = [obj for obj in algos.asyn_fluid.asyn_fluidc(connected_communities[0],10)]
    timeStart += time.time()
    result += mod_score(result_t,connected_communities[0])
    timeStart2 -= time.time()
    result_t = [obj for obj in alternate_async_flow(connected_communities[0],10)]
    timeStart2 += time.time()
    result2 += mod_score(result_t,connected_communities[0])
  totalScoreOriginal[k] = result/numTrials
  totalScoreNew[k] = result2/numTrials
  totalTimeOriginal[k] = timeStart/numTrials
  totalTimeNew[k] = timeStart2/numTrials
print(totalScoreOriginal)
print(totalScoreNew)
print(totalTimeOriginal)
print(totalTimeNew)

{1: 0.849865515332184, 2: 0.8560053376592021, 4: 0.8337211281096079, 8: 0.8380457733074562, 16: 0.8483339287155796, 32: 0.859093971650073, 64: 0.846213186802194, 128: 0.8532197168607026, 256: 0.8465364034428189, 512: 0.8534883835420928}
{1: 0.8460518727403272, 2: 0.846303605466686, 4: 0.8530868312877853, 8: 0.839978756024679, 16: 0.8565606262627022, 32: 0.8369572762321403, 64: 0.8500865093061782, 128: 0.8526771720206354, 256: 0.8515831276362018, 512: 0.8522494277226863}
{1: 1.5884071588516235, 2: 1.5331826210021973, 4: 1.702642798423767, 8: 1.6950269937515259, 16: 1.4934841394424438, 32: 1.6914092302322388, 64: 2.0396759510040283, 128: 2.056843400001526, 256: 2.5225762128829956, 512: 1.3409879207611084}
{1: 1.653939962387085, 2: 1.861702799797058, 4: 1.5613210201263428, 8: 1.669481635093689, 16: 1.5409210920333862, 32: 1.5442808866500854, 64: 1.737174391746521, 128: 2.2227230072021484, 256: 1.8612662553787231, 512: 1.715139627456665}


In [0]:
import random
randomAllocation = []
for i in range(0,10):
  randomAllocation.append(set())
print(randomAllocation)
for i in range(len(connected_communities[0].nodes)):
  randomAllocation[random.randint(0,9)].add(i)
print(randomAllocation)
print(mod_score(randomAllocation,connected_communities[0]))

[set(), set(), set(), set(), set(), set(), set(), set(), set(), set()]
[{2050, 6148, 8200, 9, 10, 2062, 6158, 8208, 6161, 20, 2069, 4118, 6164, 8221, 8223, 8224, 8225, 4130, 4133, 8233, 2094, 47, 48, 6196, 4149, 4150, 6199, 56, 57, 6201, 59, 60, 2109, 4156, 4157, 2112, 65, 6210, 8263, 4169, 2125, 84, 2133, 4180, 6228, 6231, 4186, 6237, 94, 95, 98, 101, 2150, 103, 104, 6245, 2154, 8295, 8300, 6255, 112, 2160, 2162, 8303, 8307, 6264, 2173, 4221, 8319, 2177, 2178, 8321, 8326, 2183, 6280, 8329, 139, 4235, 2190, 6289, 147, 2195, 4247, 6295, 6296, 8346, 2203, 4252, 157, 2205, 6137, 160, 4256, 4257, 6303, 8350, 6313, 170, 2218, 4266, 4268, 4272, 179, 180, 8373, 183, 6328, 6329, 2234, 187, 4283, 8378, 8380, 191, 4287, 8381, 8387, 2251, 4300, 4304, 2259, 4308, 6356, 8403, 8404, 6360, 8410, 219, 220, 6363, 8412, 225, 4328, 8426, 235, 2285, 239, 2289, 4337, 8437, 6391, 2298, 8442, 6398, 2303, 2305, 8450, 4357, 264, 266, 2314, 4363, 6413, 2318, 272, 273, 6417, 6423, 6425, 4378, 6429, 4382, 6432, 6

# Messing around with other flow algos

In [0]:
#Modifed from https://networkx.github.io/documentation/stable/_modules/networkx/algorithms/community/asyn_fluid.html#asyn_fluidc
def signaling(G):
    iters = 4
    vertices = list(G)
    result = {}
    for vertex in vertices:
      weights = {vertex: 1}
      for _ in range(iters):
        newweights = {}
        #Push weight to neighbors
        for v in weights.keys():
          newweights[v] = newweights.get(v,0) + weights[v]
          for neighbor in G[v].keys():
            newweights[neighbor] = newweights.get(neighbor,0) + weights[v]
        weights = newweights
      result[vertex] = weights
    return result
temp_result = signaling(G)
from sklearn.cluster import KMeans
from scipy.sparse import lil_matrix
trial = lil_matrix((max(G.nodes()), max(G.nodes()))) #The matrix holding the values
for i in temp_result.keys():
  for j in temp_result[i].keys():
    trial[i-1,j-1] = temp_result[i][j]

# Example K Means

In [0]:
import time
clusters = 8
start_time = time.time()
kmeans = KMeans(n_clusters=clusters,precompute_distances=True).fit(reduced)
print(time.time() - start_time)
  
#https://stackoverflow.com/questions/43828704/how-to-cluster-sparse-data-using-sklearn-kmeans

2.1521713733673096


In [0]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
clusters = 1000
svd = TruncatedSVD(n_components=clusters*3, n_iter=20, random_state=42)
svd.fit(trial)
clustering = AgglomerativeClustering(n_clusters=clusters).fit(reduced)

In [0]:
partition = []
temp = []
for label in range(clusters):
  indices = [i+1 for i, x in enumerate(clustering.labels_) if x == label]
  partition.append(set(indices))
  temp.append(len(indices))
temp.sort(reverse=True)
print(temp)

[4206, 202, 87, 75, 59, 45, 44, 41, 39, 39, 39, 39, 39, 38, 34, 34, 33, 32, 31, 31, 29, 29, 28, 28, 27, 27, 25, 25, 24, 24, 24, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 20, 20, 20, 20, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

In [0]:
print(partition)
print(mod_score(partition,G))
partition2 = []
for p in partition:
  partition2.append(set(p))
print(partition2)

[{1024, 2048, 1026, 1027, 1028, 1029, 2054, 7, 9, 10, 12, 13, 18, 20, 22, 27, 2245, 33, 34, 2082, 39, 2599, 553, 2600, 2602, 2603, 46, 559, 1582, 49, 561, 51, 1587, 53, 566, 55, 2045, 7032, 1594, 59, 60, 61, 63, 64, 65, 66, 1599, 68, 583, 585, 1103, 2128, 1105, 594, 1107, 84, 85, 597, 598, 1108, 601, 1112, 1113, 2133, 1629, 1630, 2138, 2140, 1633, 1634, 1635, 2142, 2143, 614, 1127, 1128, 2146, 618, 1131, 1643, 1133, 2149, 623, 624, 2152, 2154, 5111, 6769, 6772, 124, 127, 130, 136, 1161, 138, 141, 2191, 144, 146, 2195, 2196, 150, 663, 665, 155, 668, 1180, 1182, 1183, 1184, 5731, 170, 173, 177, 179, 691, 695, 696, 2744, 187, 188, 701, 191, 703, 5737, 194, 707, 197, 198, 199, 711, 201, 712, 713, 715, 205, 206, 211, 212, 214, 215, 219, 5340, 222, 224, 736, 228, 230, 1254, 235, 238, 240, 242, 244, 246, 248, 250, 251, 1802, 1803, 1805, 1806, 4367, 4369, 4370, 1811, 3348, 3860, 4371, 282, 283, 284, 288, 291, 817, 824, 315, 318, 1397, 856, 1370, 860, 3423, 864, 869, 1381, 1382, 360, 5992, 362,

# Actual Performance Checker, Checks the Modularity of Graph given list of sets of nodes in communities and G
- IE [{1,2,4},{3,5}] , G

In [0]:
#Modularity perf checker
def mod_score(communities,G):
  total_sum = 0
  m = len(G.edges())
  for community in communities:
    summation = len(G.subgraph(community).edges())
    totaldegree=sum(n for _, n in G.degree(community)) 
    total_sum += ((summation/m)-(totaldegree/(2*m))**2)
  return total_sum

# Prep for Swam Algo



In [0]:
import tensorflow as tf
import numpy as np
import networkx as nx
import math

from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [0]:
import re
lineFinder = re.compile('(\d+\s+\d+.*\n)')
# Options com-youtube.ungraph.txt  ca-HepTh.txt roadNet-CA.txt
#with open('/content/drive/My Drive/ca-HepTh.txt', 'r') as f:
with open('/content/drive/My Drive/ca-HepTh.txt', 'r') as f:
   file = f.read()
singleLines = list(map(lambda x: np.fromstring(x, dtype=int, sep=' '), lineFinder.findall(file)))
G = nx.from_edgelist(np.column_stack(singleLines).T)
G = nx.convert_node_labels_to_integers(G)
print("Data impoted")

G = sorted(list(nx.connected_component_subgraphs(G)),key=lambda x: -1*len(x.nodes()))[0];

from scipy.sparse import lil_matrix
adjacency_matrix_R = lil_matrix((len(G.nodes()), len(G.nodes())))
#diag_matrix = lil_matrix((len(G.nodes()), len(G.nodes())))
normal_laplacian = lil_matrix((len(G.nodes()), len(G.nodes())))
#print(adjacency_matrix.shape)

G = nx.convert_node_labels_to_integers(G)
for edge in G.edges():
  adjacency_matrix_R[edge[0],edge[1]] = 1
  adjacency_matrix_R[edge[1],edge[0]] = 1

print("Adjacency_matrix prepared")

Data impoted
Adjacency_matrix prepared


# Swarm Algorithm Initialization



In [0]:
import random

popsize = 20

Kval=64
number_of_edge = G.number_of_edges()
number_of_nodes = G.number_of_nodes()
pop=[]

class Particle:
    def __init__(self, number_of_nodes):
        self.X = np.array(list(range(1,number_of_nodes+1))) #X is from 1 to number of nodes. Numpy array
        self.V = np.zeros(number_of_nodes)                         #V is number of nodes of 0s. Numpy array
        
def Reordering(X):
    counter=1
    x_copy = X
    ordered_X={}
    for i in range(0, len(X)):
        if X[i] in ordered_X:
          X[i] = ordered_X[X[i]]
        else:
          ordered_X[X[i]] = counter
          X[i] = counter
          counter+=1
    return x_copy.astype(int)
        
def fitness(X):
  result = {}
  cnt = []
  for i in range(number_of_nodes):
    if X[i]-1 in result:
      result[X[i]-1].add(i)
    else:
      result[X[i]-1] = set({X[i]-1})
  return mod_score(result.values(),G)

    
def mod_score(communities,G):
  total_sum = 0
  m = len(G.edges())
  for community in communities:
    summation = len(G.subgraph(community).edges())
    totaldegree=sum(n for _, n in G.degree(community)) 
    total_sum += ((summation/m)-(totaldegree/(2*m))**2)
  return total_sum


iter = 5
best_particle_index=0
Pbest=[]
Pbest_Q=[]
Gbest_Q=-np.inf


for l in range(popsize):
  pop.append(Particle(number_of_nodes))
  for i in range(number_of_nodes):
    pop[l].X[i] = random.randint(1,Kval)

neighbor_list_dict = dict()
for k in range(0, number_of_nodes):
  neighbor_list_dict[k]=np.array(list(nx.classes.function.neighbors(G,k))) 
  
for i in range(0, len(pop)):  
  print(i)
  for k in range(0, int(0.2*number_of_nodes)):
      random_index=random.randint(0,number_of_nodes-1)
      label=pop[i].X[random_index]
      neighbor_list=neighbor_list_dict[random_index]
      for neighbor in neighbor_list:
         pop[i].X[neighbor]=label
  pop[i].X = Reordering(pop[i].X)                    
  Pbest.append(pop[i].X)                                                                  
  Q=fitness(pop[i].X)
  Pbest_Q.append(Q)
  if(Q>Gbest_Q):                                                        #??no need to compare if all same
      best_particle_index=i
      Gbest_Q=Q
Gbest=pop[best_particle_index].X
print("Initilization Finished")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Initilization Finished


# Swarm Algorithm Main

In [0]:
from itertools import combinations
import collections
import time
  
def Substract(particle,Best):
    result=np.empty(number_of_nodes).astype(int)
    for k in range(0, number_of_nodes):
        if Best[k] == particle.X[k]:
            result[k] = 0
        else:
           result[k] = 1
    return result

def Addition(velocity1,velocity2):
    result=((velocity1+velocity2)>=1).astype(int)
    return result


def Multiply(X,V):
    result = np.empty(number_of_nodes)
    num_degrees = collections.Counter()
    num_edges = collections.Counter()
    for i in range(number_of_nodes):
      num_degrees[X[i]]+=G.degree(i)
    for a,b in G.edges():
      if X[a] == X[b]:
        num_edges[X[a]]+=1
    for i in range(0, number_of_nodes):
        if V[i]==0:
            result[i] = X[i]
        else:
            neighbor_list=neighbor_list_dict[i]
            neighbor_size = neighbor_list.shape[0]
            del_Q_Best=-np.inf
            best_label=0
            total_sum = num_edges[X[i]]/number_of_edge-(num_degrees[X[i]]/(2*number_of_edge))**2
            changed = 0
            for k in range(neighbor_size):
               if X[k] == X[i]:
                changed+=1
            total_sum4 = (num_edges[X[i]] - changed)/number_of_edge-((num_degrees[X[i]]-G.degree(i))/(2*number_of_edge))**2
            for k in range(0, neighbor_size):
                #if i and neighbor_list k are same class, then the change would be zero
                del_Q = 0
                if X[i] != X[neighbor_list[k]]:
                  changed = 0
                  for j in range(neighbor_size):
                    if X[neighbor_list[j]] == X[neighbor_list[k]]:
                      changed+=1
                  total_sum2 = num_edges[X[neighbor_list[k]]]/number_of_edge-(num_degrees[X[neighbor_list[k]]]/(2*number_of_edge))**2
                  total_sum3 = (num_edges[X[neighbor_list[k]]] + changed)/number_of_edge-((num_degrees[X[neighbor_list[k]]]+G.degree(i))/(2*number_of_edge))**2
                  del_Q=total_sum4+total_sum3-total_sum2-total_sum
                if(del_Q>del_Q_Best):
                  best_label=X[neighbor_list[k]]
                  del_Q_Best=del_Q  
            result[i]=best_label
    return result.astype(int)

t=0
tmax=200
omega=0.7298
c1=1.4961
c2=1.4961
start = time.time()
times=[]
print("t               Ave.currQ                        FInal GbestQ \n")
while t<tmax:
    #print( '{}{}'.format("t=", t))
    print(t,end="        ")
    count = 0
    for i in range(0, len(pop)):
            pop[i].V=Addition(c1*np.random.rand()*Substract(pop[i], Pbest[i])+c2*np.random.rand()*Substract(pop[i],Gbest),omega*pop[i].V)
            pop[i].X = Multiply(pop[i].X,pop[i].V)
            pop[i].X = Reordering(pop[i].X)
            curr_Q=fitness(pop[i].X)
            Pbest_Q_val=Pbest_Q[i]
            count+=curr_Q
            if(curr_Q>Pbest_Q_val):
                Pbest[i]=pop[i].X
                Pbest_Q[i]=curr_Q
            if (Pbest_Q_val > Gbest_Q):     
                Gbest=pop[i].X                                                                             
                Gbest_Q=Pbest_Q_val
    #print( '{}{}'.format("Ave.currQ: ", count/len(pop)))
    print(count/len(pop),end="          ")
    #print(Pbest_Q_val)
    #print('{}{}'.format("GbestQ: ", Gbest_Q,end=" "))
    print(Gbest_Q,end="\n")
    #print({}{}'.format("Total number of comunity: of  ", Gbest_Qmax(Pbest[0]))
    if((t+1)%25==0):
      times.append(int((time.time()-start)))
               
    t+=1

end = time.time()
print("--FINISHED--")
print('{}{}'.format("FInal GbestQ: ", Gbest_Q))
print('{}{}'.format("Total Execution Time:  ", end - start))
print('{}{}'.format("Execution Time:  ", end - start))
for i in range(len(times)):
    print('{}{}'.format(str(25*(i+1))+"     ", times[i]))


t               Ave.currQ                        FInal GbestQ 

0        0.2624536213402562          0.34004728843141807
1        0.24896196944466054          0.34004728843141807
2        0.24214297761470432          0.34004728843141807
3        0.2514483970567446          0.34004728843141807
4        0.2455510697891333          0.34004728843141807
5        0.25500169841270093          0.34004728843141807
6        0.25819757908194313          0.34004728843141807
7        0.24885573843450043          0.34004728843141807
8        0.2467961593862996          0.34004728843141807
9        0.25195928382509036          0.34004728843141807
10        0.25575690559596403          0.34004728843141807
11        0.2593665128695325          0.34004728843141807
12        0.262174662163602          0.34004728843141807
13        0.2681548405026018          0.34004728843141807
14        0.26631816810829345          0.34004728843141807
15        0.2689466968350944          0.34004728843141807
16        0

In [0]:
print(Gbest_Q)


0.35562021313560377


In [0]:
result = []
for j in range(1,int(9987)+1):
  result2 = set()
  for i in range(9877):
    if(Gbest[i]==j):
      result2.add(i)
  if(len(result2)>0):
    result.append(result2)
print(mod_score(result,G))

0.43394020568881975


In [0]:
while t<tmax:
    #Gbest_Q=fitness(Gbest,number_of_nodes,number_of_edge)
    for i in range(0, len(pop)):
            
            pop[i].V=Addition(c1*np.random.rand()*Substract(pop[i], Pbest[i], number_of_nodes)+c2*np.random.rand()*Substract(particle=pop[i], Best=Gbest, number_of_nodes=number_of_nodes),omega*pop[i].V)
            pop[i].X=Multiply(pop[i].X,pop[i].V)
            Reordering(pop[i].X, number_of_nodes)
            curr_Q=fitness(pop[i].X, number_of_nodes, number_of_edge)
            #if i==0:
            # Pbest_Q=fitness(Pbest[i], number_of_nodes, number_of_edge)               #?? can be saved once @
            Pbest_Q_val=Pbest_Q[i]
            if(curr_Q>Pbest_Q_val):
                Pbest[i]=pop[i].X
                Pbest_Q[i]=curr_Q
            if (Pbest_Q_val > Gbest_Q):     
                Gbest=pop[i].X                                                                             #?? @
                Gbest_Q=Pbest_Q_val
                
    t+=1
    #print(t,end="")
    
print(Gbest_Q)

1-0.21915500553840647


In [0]:
print(Pbest_Q)

[-1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09, -1.1096381626188755e-09]


In [0]:
adjacency_matrix[2,4]

-1.0